# Tutorial 3: Introspect Compose

## Overview

While constructing and using a flow, it's convenient to introspect the inner architecture of the flow to understand how it work and to ensure that we plug-n-play the right component into the flow. This capability can prove even more usefulness for those who don't build the flow and just use those that are packaged and distributed by other people.

Any flow constructed with `theflow.Compose` allow users to:

- View values of all current nodes and params.
- View the definition of any nodes and params.
- Check if any object is compatible with the flow and can be plugged into the flow.
- Visualize the flow architecture.

Suppose we have a flow `Plus` that consists of component `Sum1` and `Sum2` as follow:

In [1]:
import yaml
from pprint import pprint

from theflow import Compose, Param, Node


class Sum1(Compose):
    a: int
    b: int = 10
    c: int = 10

    def run(self) -> int:
        return self.a + self.b + self.c


class Sum2(Compose):
    a: int

    def run(self, a, b: int, *args, **kwargs) -> int:
        return self.a + a + b


class Plus(Compose):
    a: int
    x: Compose
    y: Compose = Node(default=Sum1, default_kwargs={"a": 100})
    m: Compose = Node(default=Sum2, default_kwargs={"a": 100})
        
    def run(self) -> int:
        x, y, m = self.x(), self.y(), self.m()
        print(f"{x=}, {y=}, {z=}")
        return x + y + z

## View values of current nodes and params

`theflow` allows viewing all values of current nodes and params with `.describe()`. This method returns a dictionary contains name and value of all nodes and params.

In [2]:
step = Plus(a=1, x=Sum1(a=10))
print(yaml.dump(step.describe(), sort_keys=False))

type: __main__.Plus
params:
  a:
    __type__: param
    default: '{{ theflow.base.empty }}'
    default_callback: null
    help: ''
    refresh_on_set: false
    strict_type: false
    depends_on: null
    no_cache: false
nodes:
  y:
    __type__: theflow.base.Node
    type: node
    default:
      type: __main__.Sum1
      params:
        b:
          __type__: param
          default: 10
          default_callback: null
          help: ''
          refresh_on_set: false
          strict_type: false
          depends_on: null
          no_cache: false
        c:
          __type__: param
          default: 10
          default_callback: null
          help: ''
          refresh_on_set: false
          strict_type: false
          depends_on: null
          no_cache: false
        a:
          __type__: param
          default: '{{ theflow.base.empty }}'
          default_callback: null
          help: ''
          refresh_on_set: false
          strict_type: false
          depends_o

## View the definition of any node and param

`theflow` allows getting the definition of a node or a param with `.specs(...)`:

In [3]:
print("Node m")
pprint(step.specs("m"))
print("-" * 10)
print("Param x.b")
pprint(step.specs("x.b"))
print("-" * 10)

Node m
{'default': <class '__main__.Sum2'>,
 'default_callback': None,
 'default_kwargs': {'a': 100},
 'depends_on': None,
 'help': '',
 'input': {'a': typing.Any, 'b': <class 'int'>},
 'no_cache': False,
 'output': <class 'int'>,
 'type': 'node'}
----------
Param x.b
{'__type__': 'param',
 'default': 10,
 'default_callback': None,
 'depends_on': None,
 'help': '',
 'no_cache': False,
 'refresh_on_set': False,
 'strict_type': False}
----------


## Check if any object is compatible with the flow

`theflow` will inspect the method `.run`'s input and output signature to determine if a component can be plug-n-play to an existing flow.

In [4]:
from typing import List

class WrongInputName(Compose):
    def run(self, x, b: int, *args, **kwargs):
        return 10

class MissingInputVariable(Compose):
    def run(self, b: int, *args, **kwargs):
        return 10

class WrongInputType(Compose):
    def run(self, a, b: List[int], *args, **kwargs):
        return 10

class WrongOutputType(Compose):
    def run(self, a, b: int, *args, **kwargs) -> str:
        return "10"

class CorrectExact(Compose):
    def run(self, a, b: int, *args, **kwargs) -> int:
        return 10

class CorrectExtraArgs(Compose):
    def run(self, a, b: int, x, *args, **kwargs) -> int:
        return 10

class CorrectMissingTypeAssumeTypeAny(Compose):
    def run(self, a, b, x, *args, **kwargs):
        return 10

print(f"{step.is_compatible('m', WrongInputName)=}")
print(f"{step.is_compatible('m', MissingInputVariable)=}")
print(f"{step.is_compatible('m', WrongInputType)=}")
print(f"{step.is_compatible('m', WrongOutputType)=}")
print(f"{step.is_compatible('m', CorrectExact)=}")
print(f"{step.is_compatible('m', CorrectExtraArgs)=}")
print(f"{step.is_compatible('m', CorrectMissingTypeAssumeTypeAny)=}")

step.is_compatible('m', WrongInputName)=False
step.is_compatible('m', MissingInputVariable)=False
step.is_compatible('m', WrongInputType)=False
step.is_compatible('m', WrongOutputType)=False
step.is_compatible('m', CorrectExact)=True
step.is_compatible('m', CorrectExtraArgs)=True
step.is_compatible('m', CorrectMissingTypeAssumeTypeAny)=True


-------